<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/tune_similarity_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall gensim -y

Found existing installation: gensim 4.1.2
Uninstalling gensim-4.1.2:
  Successfully uninstalled gensim-4.1.2


In [2]:
!pip install gensim

  Using cached gensim-4.1.2-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)


In [3]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

In [6]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy' -O input_seq.txt


--2021-11-14 16:59:30--  https://docs.google.com/uc?export=download&id=1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy
Resolving docs.google.com (docs.google.com)... 142.250.136.113, 142.250.136.138, 142.250.136.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.136.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7eh4qrhcf30k0u6qgjdn1755spdga0vd/1636909125000/14329102864480165501/*/1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy?e=download [following]
--2021-11-14 16:59:30--  https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7eh4qrhcf30k0u6qgjdn1755spdga0vd/1636909125000/14329102864480165501/*/1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy?e=download
Resolving doc-10-4c-docs.googleusercontent.com (doc-10-4c-docs.googleusercontent.com)... 74.125.69.132, 2607:f8b0:4001:c08::84
Connecting to doc-10-4c-docs.googleusercontent.com (doc-10-4

Major Triads, Minor Triads and Dom7 chords:

In [7]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q' -O input_meta.csv

--2021-11-14 16:59:30--  https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q
Resolving docs.google.com (docs.google.com)... 209.85.147.113, 209.85.147.139, 209.85.147.100, ...
Connecting to docs.google.com (docs.google.com)|209.85.147.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/igfo17mpj5ss2hurcs7jrqu3f2koafn3/1636909125000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download [following]
--2021-11-14 16:59:31--  https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/igfo17mpj5ss2hurcs7jrqu3f2koafn3/1636909125000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download
Resolving doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.googleusercontent.com)... 74.125.69.132, 2607:f8b0:4001:c08::84
Connecting to doc-14-4c-docs.googleusercontent.com (doc-14-4c-do

In [8]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd' -O input_seq.txt

--2021-11-14 16:59:31--  https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd
Resolving docs.google.com (docs.google.com)... 142.250.136.138, 142.250.136.113, 142.250.136.101, ...
Connecting to docs.google.com (docs.google.com)|142.250.136.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/u8dmcre15830iui9s0k3827so7ijcrd6/1636909125000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download [following]
--2021-11-14 16:59:32--  https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/u8dmcre15830iui9s0k3827so7ijcrd6/1636909125000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download
Resolving doc-08-4c-docs.googleusercontent.com (doc-08-4c-docs.googleusercontent.com)... 74.125.69.132, 2607:f8b0:4001:c08::84
Connecting to doc-08-4c-docs.googleusercontent.com (doc-08-4

Meta Data:

In [9]:
path_to_file = '/content/input_seq.txt'

with open(path_to_file) as f: 
  lines = f.read().splitlines()

In [10]:
data = [line.split(' ') for line in lines]

### Meta Data

In [11]:
meta = pd.read_csv('input_meta.csv', sep='\t')
meta.columns

Index(['id', 'file_name', 'title', 'composer', 'year', 'year_truncated',
       'tonality', 'tune_key', 'tune_mode', 'structure', 'num_bars',
       'time_signature', 'cycle_fifths_order', 'style', 'musicbrainz_id',
       'wikidata_id', 'wikidata_allmusic', 'wiki_link', 'wikidata_description',
       'lyricist'],
      dtype='object')

In [12]:
titles = meta.loc[:, ['id', 'title', 'tune_mode']].drop_duplicates()
titles.set_index('id', inplace=True)

In [13]:
assert len(data) == len(titles)
print(len(data))

1812


In [14]:
titles_dict = titles.to_dict()
index_to_title = titles_dict['title']
index_to_title[1170]

'These Foolish Things'

In [15]:
title_to_index = {value: index for index, value in index_to_title.items()}
title_to_index['These Foolish Things']

1170

# Visualize Chord N-Grams



In [16]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [17]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

In [18]:
data_ngrams = list_to_ngrams(data, 3)

In [19]:
def raw_chords_to_df(tunes):
  tunes_chords = [item for tune in tunes for item in tune]
  counts = Counter(tunes_chords)
  df = pd.DataFrame(counts.items(),
                    columns=['chord', 'count']).sort_values(by='count', ascending=False)

  return df

In [20]:
df_chords = raw_chords_to_df(data_ngrams)
df_chords.head(10)

,chord,count
14,Dm7-G7-C,3344
75,G7-C-C,1630
132,A7-Dm7-G7,1506
37,Em7-A7-Dm7,1132
410,C-C-C,1059
179,D7-G7-C,838
85,G7-G7-C,826
71,Am7-Dm7-G7,663
74,C-G7-C,638
6,Gm7-C7-F,615


In [21]:
df_chords = pd.DataFrame.from_dict(df_chords)
df_chords.sort_values(by=['count'], ascending=False, inplace=True)
df_chords_top = df_chords.query('count > 100')

fig = px.bar(df_chords_top, x='chord', y='count', log_y=True)
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

2021-11-14 16:59:32,795 : INFO : NumExpr defaulting to 2 threads.


# List Tunes for Test Data

In [22]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               #"Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
               "Sweet Sue, Just You": "Bye Bye Blackbird", # Bridge same
               "These Foolish Things": "Blue Moon", # first 8 bars same
               
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_index[reference], title_to_index[compare]])
  

In [23]:
contrafacts_test[:5]

[[0, 251], [3, 1527], [30, 63], [80, 1527], [104, 1372]]

# TF-IDF

## Build Model

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
tfidf_ngrams = [1, 2, 3, 4]

tfidf_model = {}
for n in tfidf_ngrams:

  data_ngrams = list_to_ngrams(data, n=n)
  data_tfidf = [" ".join(i) for i in data_ngrams]

  tfidf_vectorizer = TfidfVectorizer()
  tfidf_model[n] = tfidf_vectorizer.fit_transform(data_tfidf)

In [26]:
tfidf_model

{1: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 2: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 3: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>,
 4: <1812x30 sparse matrix of type '<class 'numpy.float64'>'
 	with 15947 stored elements in Compressed Sparse Row format>}

## Test of the Contrafacts tunes

In [27]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2']
for c in tfidf_model.keys():
  cols.append(f"cosval_ngram{c}")
df_eval_tfidf = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  for ngram_size, tfidf_m in tfidf_model.items():
    row.append(cosine_similarity(tfidf_m[tune1], tfidf_m[tune2])[0][0])
  
  df_eval_tfidf.loc[len(df_eval_tfidf)] = row

df_eval_tfidf

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4
0,26-2,Confirmation,0.568533,0.564252,0.574305,0.576989
1,52nd Street Theme,I Got Rhythm,0.832439,0.832439,0.825817,0.817865
2,Ablution,All The Things You Are,0.886735,0.877070,0.879810,0.878596
3,Anthropology,I Got Rhythm,0.777413,0.777413,0.784389,0.783179
4,Ballade,As Long As I Live,0.634909,0.643877,0.647502,0.641000
5,Bright Mississippi,Sweet Georgia Brown,0.970778,0.969760,0.968172,0.965051
6,C.T.A.,I Got Rhythm,0.698455,0.695247,0.704302,0.709436
7,Celia,I Got Rhythm,0.505318,0.489912,0.486457,0.473649
8,Cottontail,I Got Rhythm,0.915579,0.915579,0.912508,0.909247
9,Countdown,Tune Up,0.744414,0.725241,0.712231,0.697034


In [28]:
print(tfidf_model[2][1171].toarray())

[[0.26713904 0.         0.45952102 0.08441284 0.         0.07890011
  0.         0.         0.         0.         0.17897608 0.
  0.         0.21980559 0.         0.51633036 0.         0.
  0.         0.         0.         0.         0.20390975 0.
  0.09783795 0.         0.5392851  0.         0.08595767 0.08839597]]


In [85]:
m = tfidf_model[1]
tfidf_cos = cosine_similarity(m)

In [122]:
from operator import itemgetter

tfidf_dict = {}
for row_index, row in enumerate(tfidf_cos):
  indices, L_sorted = zip(*sorted(enumerate(row), key=itemgetter(1), reverse=True))
  
  # do not report self-similiarity, therefore remove the reference tune
  remove_index = indices.index(row_index)
  
  indices = list(indices)
  indices.pop(remove_index)
  L_sorted = list(L_sorted)
  L_sorted.pop(remove_index)

  tfidf_dict[row_index] = {}
  tfidf_dict[row_index]['tune_indices'] = indices[:10]
  tfidf_dict[row_index]['score'] = [round(num, 3) for num in L_sorted[:10]] 

tfidf_dict

{0: {'score': [0.764,
   0.751,
   0.738,
   0.726,
   0.72,
   0.718,
   0.7,
   0.676,
   0.674,
   0.671],
  'tune_indices': [425, 192, 769, 1215, 20, 262, 549, 356, 861, 991]},
 1: {'score': [0.811,
   0.792,
   0.756,
   0.734,
   0.719,
   0.712,
   0.705,
   0.684,
   0.679,
   0.675],
  'tune_indices': [1005, 338, 716, 1194, 852, 66, 448, 214, 1202, 808]},
 2: {'score': [0.834,
   0.822,
   0.809,
   0.807,
   0.802,
   0.798,
   0.791,
   0.775,
   0.775,
   0.767],
  'tune_indices': [282, 1326, 1189, 315, 913, 153, 999, 441, 912, 137]},
 3: {'score': [0.979,
   0.965,
   0.962,
   0.959,
   0.943,
   0.935,
   0.934,
   0.933,
   0.917,
   0.915],
  'tune_indices': [516, 391, 835, 651, 457, 92, 582, 318, 631, 132]},
 4: {'score': [0.747,
   0.742,
   0.731,
   0.713,
   0.663,
   0.661,
   0.642,
   0.639,
   0.633,
   0.63],
  'tune_indices': [596, 107, 1416, 1574, 471, 1575, 672, 899, 1295, 184]},
 5: {'score': [0.881,
   0.864,
   0.863,
   0.853,
   0.848,
   0.831,
   0.

In [123]:
tfidf_file = 'tfidf_cossim.pickle'
with open(tfidf_file, 'wb') as f:
      # Pickle the 'data' dictionary using the highest protocol available.
      pickle.dump(tfidf_dict, f, pickle.HIGHEST_PROTOCOL)

from google.colab import files
files.download(tfidf_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Doc2Vec Embeddings

## Read and Tag Data

In [29]:
import smart_open

def read_corpus(fname, ngram_size, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = ngrams(line.split(), ngram_size, '-')
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

## Doc2Vec Hyperparameters

In [30]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

In [31]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [32]:
import multiprocessing

model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': ['pv-dm', 
                                  'concat',
                                  'pv-dbow'
                                  ], 
                      'window': [
                                 2, 
                                 3, 
                                 #4, 
                                 #10
                                 ],
                      'sample': [0.1],
                      'ngrams': [
                                 1, 
                                 2, 
                                 #3, 
                                 #4
                                 ],
                     }
              }


common_kwargs = dict(
    vector_size=100, 
    epochs=20, 
    min_count=2,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=10, 
    hs=0,
)

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 0 if variant == 'pv-dbow' else 1
        concat = 1 if variant ==  'concat' else 0

        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(path_to_file, ngram_size=ngram_size))

            # evaluate different window sizes
            for window in params['window']:

                # evaluate the effect of different sample sizes
                for sample in params['sample']:

                    # calculate the model
                    if algo == 'd2v':
                        print(f"Model: {variant}, dm={dm}, dm_concat={concat}; ngrams={ngram_size}, window={window}, sample={sample}")
                        print(f"Corpus: {train_corpus[0]}")
                        model = gensim.models.doc2vec.Doc2Vec(**common_kwargs,
                                                              dm = dm,
                                                              dm_concat = concat,
                                                              )
                        model.build_vocab(train_corpus)
                        model.train(train_corpus, 
                                    total_examples=model.corpus_count, 
                                    epochs=model.epochs)

                    else:
                        model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                    file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    model.save(file_name + '.model')

                    file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    with open(file_name + '.train', 'wb') as f:
                          # Pickle the 'data' dictionary using the highest protocol available.
                          pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

2021-11-14 16:59:35,321 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n10,w5,mc2,t2)', 'datetime': '2021-11-14T16:59:35.320930', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-14 16:59:35,323 : INFO : collecting all words and their counts
2021-11-14 16:59:35,325 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-14 16:59:35,348 : INFO : collected 48 word types and 1812 unique tags from a corpus of 1812 examples and 82790 words
2021-11-14 16:59:35,349 : INFO : Creating a fresh vocabulary
2021-11-14 16:59:35,353 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=2 retains 48 unique words (100.0%% of original 48, drops 0)', 'datetime': '2021-11-14T16:59:35.353126', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=2, sample=0.1
Corpus: TaggedDocument(['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], [0])


2021-11-14 16:59:35,676 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 16:59:35,687 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 16:59:35,693 : INFO : EPOCH - 1 : training on 82790 raw words (84602 effective words) took 0.3s, 303623 effective words/s
2021-11-14 16:59:35,973 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 16:59:35,987 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 16:59:35,988 : INFO : EPOCH - 2 : training on 82790 raw words (84602 effective words) took 0.3s, 295864 effective words/s
2021-11-14 16:59:36,252 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 16:59:36,264 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 16:59:36,267 : INFO : EPOCH - 3 : training on 82790 raw words (84602 effective words) took 0.3s, 314306 effective words/s
2021-11-14 16:59:36,526 : INFO : worker thread finished

Model: pv-dm, dm=1, dm_concat=0; ngrams=1, window=3, sample=0.1
Corpus: TaggedDocument(['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], [0])


2021-11-14 16:59:41,174 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 16:59:41,189 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 16:59:41,191 : INFO : EPOCH - 1 : training on 82790 raw words (84602 effective words) took 0.3s, 297351 effective words/s
2021-11-14 16:59:41,441 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 16:59:41,455 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 16:59:41,457 : INFO : EPOCH - 2 : training on 82790 raw words (84602 effective words) took 0.3s, 332196 effective words/s
2021-11-14 16:59:41,730 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 16:59:41,742 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 16:59:41,744 : INFO : EPOCH - 3 : training on 82790 raw words (84602 effective words) took 0.3s, 302627 effective words/s
2021-11-14 16:59:41,996 : INFO : worker thread finished

...

2021-11-14 17:00:35,048 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:35,062 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:35,065 : INFO : EPOCH - 1 : training on 82790 raw words (84602 effective words) took 0.2s, 417414 effective words/s
2021-11-14 17:00:35,219 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:35,230 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:35,231 : INFO : EPOCH - 2 : training on 82790 raw words (84602 effective words) took 0.2s, 542081 effective words/s
2021-11-14 17:00:35,389 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:35,401 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:35,404 : INFO : EPOCH - 3 : training on 82790 raw words (84602 effective words) took 0.2s, 520371 effective words/s
2021-11-14 17:00:35,575 : INFO : worker thread finished

Model: pv-dbow, dm=0, dm_concat=0; ngrams=2, window=2, sample=0.1
Corpus: TaggedDocument(['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C'], [0])


2021-11-14 17:00:38,794 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:38,805 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:38,806 : INFO : EPOCH - 1 : training on 80978 raw words (82470 effective words) took 0.2s, 386764 effective words/s
2021-11-14 17:00:39,012 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:39,018 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:39,021 : INFO : EPOCH - 2 : training on 80978 raw words (82470 effective words) took 0.2s, 400117 effective words/s
2021-11-14 17:00:39,216 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:39,221 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:39,225 : INFO : EPOCH - 3 : training on 80978 raw words (82470 effective words) took 0.2s, 420639 effective words/s
2021-11-14 17:00:39,412 : INFO : worker thread finished

Model: pv-dbow, dm=0, dm_concat=0; ngrams=2, window=3, sample=0.1
Corpus: TaggedDocument(['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C'], [0])


2021-11-14 17:00:43,135 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:43,145 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:43,146 : INFO : EPOCH - 1 : training on 80978 raw words (82470 effective words) took 0.2s, 384378 effective words/s
2021-11-14 17:00:43,335 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:43,342 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:43,345 : INFO : EPOCH - 2 : training on 80978 raw words (82470 effective words) took 0.2s, 438276 effective words/s
2021-11-14 17:00:43,534 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:43,543 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:43,544 : INFO : EPOCH - 3 : training on 80978 raw words (82470 effective words) took 0.2s, 432490 effective words/s
2021-11-14 17:00:43,736 : INFO : worker thread finished

### Trial with Models according to the Original Paper 

In [33]:
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

train_corpus = list(read_corpus(path_to_file, ngram_size=1))

common_kwargs = dict(
    vector_size=100, 
    epochs=50, 
    min_count=2,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=5, 
    hs=0,
)

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, **common_kwargs),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, window=10, alpha=0.05, comment='alpha=0.05', **common_kwargs),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, window=5, **common_kwargs),
]

for model in simple_models:
    model.build_vocab(train_corpus)
    model.train(train_corpus, 
                total_examples=model.corpus_count, 
                epochs=model.epochs)

    print("%s vocabulary scanned & state initialized" % model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

2021-11-14 17:00:47,176 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n5,mc2,t2)', 'datetime': '2021-11-14T17:00:47.176198', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-14 17:00:47,179 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n5,w10,mc2,t2)', 'datetime': '2021-11-14T17:00:47.179458', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-14 17:00:47,180 : INFO : using concatenative 1100-dimensional layer1
2021-11-14 17:00:47,183 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/c,d100,n5,w5,mc2,t2)', 'datetime': '2021-11-14T17:00:47.183644', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event

Doc2Vec(dbow,d100,n5,mc2,t2) vocabulary scanned & state initialized


2021-11-14 17:00:55,296 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:55,306 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:55,307 : INFO : EPOCH - 1 : training on 82790 raw words (84602 effective words) took 0.3s, 283908 effective words/s
2021-11-14 17:00:55,555 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:55,569 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:55,571 : INFO : EPOCH - 2 : training on 82790 raw words (84602 effective words) took 0.3s, 331651 effective words/s
2021-11-14 17:00:55,835 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:00:55,845 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:00:55,846 : INFO : EPOCH - 3 : training on 82790 raw words (84602 effective words) took 0.3s, 318375 effective words/s
2021-11-14 17:00:56,141 : INFO : worker thread finished

Doc2Vec(dm/m,d100,n5,w10,mc2,t2) vocabulary scanned & state initialized


2021-11-14 17:01:08,854 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:01:08,864 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:01:08,866 : INFO : EPOCH - 1 : training on 82790 raw words (84602 effective words) took 0.3s, 294801 effective words/s
2021-11-14 17:01:09,135 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:01:09,148 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:01:09,149 : INFO : EPOCH - 2 : training on 82790 raw words (84602 effective words) took 0.3s, 309154 effective words/s
2021-11-14 17:01:09,422 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-11-14 17:01:09,431 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-11-14 17:01:09,433 : INFO : EPOCH - 3 : training on 82790 raw words (84602 effective words) took 0.3s, 308259 effective words/s
2021-11-14 17:01:09,709 : INFO : worker thread finished

Doc2Vec(dm/c,d100,n5,w5,mc2,t2) vocabulary scanned & state initialized


In [34]:
!pip install testfixtures

In [35]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

2021-11-14 17:01:24,727 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-11-14 17:01:24,729 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2021-11-14 17:01:24,732 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)", 'datetime': '2021-11-14T17:01:24.732091', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}


In [36]:
models_by_name

OrderedDict([('Doc2Vec(dbow,d100,n5,mc2,t2)',
              <gensim.models.doc2vec.Doc2Vec at 0x7f2d464cacd0>),
             ('Doc2Vec(dm/m,d100,n5,w10,mc2,t2)',
              <gensim.models.doc2vec.Doc2Vec at 0x7f2d464cadd0>),
             ('Doc2Vec(dm/c,d100,n5,w5,mc2,t2)',
              <gensim.models.doc2vec.Doc2Vec at 0x7f2d43770c90>),
             ('dbow+dmm',
              <gensim.test.test_doc2vec.ConcatenatedDoc2Vec at 0x7f2d44246d50>),
             ('dbow+dmc',
              <gensim.test.test_doc2vec.ConcatenatedDoc2Vec at 0x7f2d44246e10>)])

In [37]:
from scipy import spatial

methods = ['Doc2Vec(dbow,d100,n5,mc2,t2)', 'Doc2Vec(dm/c,d100,n5,w5,mc2,t2)', 'dbow+dmm', 'dbow+dmc']

# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'score_dbow', 'score_concat', 'score_dbow+dmm', 'score_dbow+dmc']

df_eval = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
    row = [index_to_title[tune1], index_to_title[tune2]]

    for m in methods:
        vec1 = models_by_name[m][tune1]
        vec2 = models_by_name[m][tune2]
      
        row.append(1 - spatial.distance.cosine(vec1, vec2))
  
    df_eval.loc[len(df_eval)] = row

df_eval

,Tune1,Tune2,score_dbow,score_concat,score_dbow+dmm,score_dbow+dmc
0,26-2,Confirmation,0.015709,-0.037501,-0.150169,-0.018276
1,52nd Street Theme,I Got Rhythm,-0.057538,-0.040416,0.097941,-0.027294
2,Ablution,All The Things You Are,0.026116,-0.018141,-0.160364,-0.010313
3,Anthropology,I Got Rhythm,0.562932,0.441198,0.298116,0.514327
4,Ballade,As Long As I Live,0.535180,0.326133,0.609716,0.455240
5,Bright Mississippi,Sweet Georgia Brown,0.847158,0.741920,0.693507,0.807677
6,C.T.A.,I Got Rhythm,0.487613,0.379639,0.332081,0.444487
7,Celia,I Got Rhythm,0.216141,-0.023863,0.124852,0.114243
8,Cottontail,I Got Rhythm,0.835487,0.661141,0.758677,0.766458
9,Countdown,Tune Up,0.637865,0.488845,0.479407,0.580208


### End of Other Trial

In [38]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}

for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    models[name] = Doc2Vec.load(file_name)

2021-11-14 17:01:25,185 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_concat_1_3_0.1.model
2021-11-14 17:01:25,190 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_concat_1_3_0.1.model.dv.* with mmap=None
2021-11-14 17:01:25,193 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_concat_1_3_0.1.model.wv.* with mmap=None
2021-11-14 17:01:25,195 : INFO : setting ignored attribute cum_table to None
2021-11-14 17:01:25,199 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_concat_1_3_0.1.model', 'datetime': '2021-11-14T17:01:25.199892', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-14 17:01:25,203 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_concat_2_2_0.1.model
2021-11-14 17:01:25,219 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_concat_2_2_0.1.model.dv.* with m

['autos_d2v_concat_1_3_0.1.model', 'autos_d2v_concat_2_2_0.1.model', 'autos_d2v_pv-dm_1_3_0.1.model', 'autos_d2v_pv-dbow_1_3_0.1.model', 'autos_d2v_concat_1_2_0.1.model', 'autos_d2v_pv-dm_1_2_0.1.model', 'autos_d2v_pv-dbow_2_2_0.1.model', 'autos_d2v_pv-dbow_1_2_0.1.model', 'autos_d2v_pv-dbow_2_3_0.1.model', 'autos_d2v_pv-dm_2_2_0.1.model', 'autos_d2v_concat_2_3_0.1.model', 'autos_d2v_pv-dm_2_3_0.1.model']
./models_doc2vec/autos_d2v_concat_1_3_0.1.model
./models_doc2vec/autos_d2v_concat_2_2_0.1.model
./models_doc2vec/autos_d2v_pv-dm_1_3_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_1_3_0.1.model
./models_doc2vec/autos_d2v_concat_1_2_0.1.model
./models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_2_2_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model
./models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model


2021-11-14 17:01:25,391 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model.dv.* with mmap=None
2021-11-14 17:01:25,395 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model.wv.* with mmap=None
2021-11-14 17:01:25,397 : INFO : setting ignored attribute cum_table to None
2021-11-14 17:01:25,428 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model', 'datetime': '2021-11-14T17:01:25.428881', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-14 17:01:25,430 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model
2021-11-14 17:01:25,441 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model.dv.* with mmap=None
2021-11-14 17:01:25,443 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dm_2_2_0

./models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model
./models_doc2vec/autos_d2v_concat_2_3_0.1.model
./models_doc2vec/autos_d2v_pv-dm_2_3_0.1.model


In [39]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

['autos_d2v_concat_2_2_0.1.train', 'autos_d2v_pv-dm_1_3_0.1.train', 'autos_d2v_pv-dbow_1_2_0.1.train', 'autos_d2v_pv-dm_1_2_0.1.train', 'autos_d2v_pv-dbow_1_3_0.1.train', 'autos_d2v_concat_1_2_0.1.train', 'autos_d2v_concat_2_3_0.1.train', 'autos_d2v_pv-dm_2_3_0.1.train', 'autos_d2v_pv-dm_2_2_0.1.train', 'autos_d2v_concat_1_3_0.1.train', 'autos_d2v_pv-dbow_2_3_0.1.train', 'autos_d2v_pv-dbow_2_2_0.1.train']
./traindata_doc2vec/autos_d2v_concat_2_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_1_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_1_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_1_2_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_1_3_0.1.train
./traindata_doc2vec/autos_d2v_concat_1_2_0.1.train
./traindata_doc2vec/autos_d2v_concat_2_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_2_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dm_2_2_0.1.train
./traindata_doc2vec/autos_d2v_concat_1_3_0.1.train
./traindata_doc2vec/autos_d2v_pv-dbow_2_3_0.1.train
./traindata_doc2vec/autos_d2v_p

In [40]:
for name, m in models.items():
  print(m)

Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dbow,d100,n10,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)
Doc2Vec(dm/c,d100,n10,w5,mc2,t2)
Doc2Vec(dm/m,d100,n10,w5,mc2,t2)


In [41]:
def compare_doc_models(models, **kwargs):

    passes = 1

    df = pd.DataFrame()
    names = []
    scores = []
    runs = []

    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        ranks = []
        second_ranks = []
        similarities = []

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
            similarities.append(sims)
            rank = [docid for docid, sim in sims].index(doc_id)
            ranks.append(rank)

            second_ranks.append(sims[1])

        counter = Counter(ranks)
        print(counter)

        tunes_sim = counter[0]
        tunes_not_sim = len(train_corpus) - counter[0]
        score = 100 * tunes_not_sim/(tunes_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in first place: {tunes_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [44]:
df_score = compare_doc_models(models)


*** Run 0
autos_d2v_concat_1_3_0.1.model
[TaggedDocument(words=['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], tags=[0]), TaggedDocument(words=['Am7', 'Am7', 'Cm7', 'Cm7', 'Eb', 'Eb', 'Em7', 'A7', 'Dm7', 'Dm7', 'Bm7', 'Bm7', 'Bbm7', 'Bbm7', 'Fm7', 'Fm7', 'E7', 'E7', 'Fm7', 'Fm7', 'C#', 'C#', 'Fm7', 'Fm7', 'C#', 'C#'], tags=[1]), TaggedDocument(words=['Am7', 'C#', 'Bm7', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Cm7', 'F7', 'Bb', 'G#m7', 'C#7', 'F#m7', 'B7', 'E', 'E', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Cm7', 'F7', 'Bb', 'G#m7', 'C#7', 'F#m7', 'B7', 'Em7', 'Em7'], tags=[2]), TaggedDocument(words=['C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C'

In [45]:
pd.pivot_table(df_score, index = 'name', values = 'score', columns='pass')

pass,0
name,
autos_d2v_concat_1_2_0.1.model,17.273731
autos_d2v_concat_1_3_0.1.model,18.322296
autos_d2v_concat_2_2_0.1.model,95.364238
autos_d2v_concat_2_3_0.1.model,95.253863
autos_d2v_pv-dbow_1_2_0.1.model,3.863135
autos_d2v_pv-dbow_1_3_0.1.model,3.752759
autos_d2v_pv-dbow_2_2_0.1.model,2.814570
autos_d2v_pv-dbow_2_3_0.1.model,2.538631
autos_d2v_pv-dm_1_2_0.1.model,40.452539


In [46]:
def compare_doc_models2(models, **kwargs):

    passes = 1

    df = pd.DataFrame()
    topn = 5

    names = []
    scores = []
    runs = []
    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        tune_sim = 0

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=topn)
            if doc_id in [key for key, value in sims]:
              tune_sim += 1

        tunes_not_sim = len(train_corpus) - tune_sim
        score = 100 * tunes_not_sim/(tune_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in top {topn} place: {tune_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [47]:
df_score2 = compare_doc_models2(models)


*** Run 0
autos_d2v_concat_1_3_0.1.model
[TaggedDocument(words=['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C'], tags=[0]), TaggedDocument(words=['Am7', 'Am7', 'Cm7', 'Cm7', 'Eb', 'Eb', 'Em7', 'A7', 'Dm7', 'Dm7', 'Bm7', 'Bm7', 'Bbm7', 'Bbm7', 'Fm7', 'Fm7', 'E7', 'E7', 'Fm7', 'Fm7', 'C#', 'C#', 'Fm7', 'Fm7', 'C#', 'C#'], tags=[1]), TaggedDocument(words=['Am7', 'C#', 'Bm7', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Cm7', 'F7', 'Bb', 'G#m7', 'C#7', 'F#m7', 'B7', 'E', 'E', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Am7', 'C#', 'Bm7', 'E7', 'Cm7', 'F7', 'Bb', 'G#m7', 'C#7', 'F#m7', 'B7', 'Em7', 'Em7'], tags=[2]), TaggedDocument(words=['C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C', 'Am7', 'Dm7', 'G7', 'C'

In [48]:
pd.pivot_table(df_score2, index = 'name', values = 'score', columns='pass')

pass,0
name,
autos_d2v_concat_1_2_0.1.model,6.346578
autos_d2v_concat_1_3_0.1.model,7.119205
autos_d2v_concat_2_2_0.1.model,93.267108
autos_d2v_concat_2_3_0.1.model,92.604857
autos_d2v_pv-dbow_1_2_0.1.model,0.331126
autos_d2v_pv-dbow_1_3_0.1.model,0.220751
autos_d2v_pv-dbow_2_2_0.1.model,0.165563
autos_d2v_pv-dbow_2_3_0.1.model,0.220751
autos_d2v_pv-dm_1_2_0.1.model,23.509934


In [49]:
import time
t = time.localtime()
timestamp = time.strftime('%Y%m%d_%H%M%S', t)
print(timestamp)

20211114_170647


In [50]:
score_filename = f'score_{timestamp}.csv'
df_score.to_csv(score_filename)

In [51]:
 !zip -r models.zip models_doc2vec/ 

updating: models_doc2vec/ (stored 0%)
updating: models_doc2vec/autos_d2v_concat_1_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_concat_2_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dm_1_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dbow_1_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_concat_1_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dm_1_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dbow_2_2_0.1.model (deflated 9%)
updating: models_doc2vec/autos_d2v_pv-dbow_1_2_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dbow_2_3_0.1.model (deflated 9%)
updating: models_doc2vec/autos_d2v_pv-dm_2_2_0.1.model (deflated 9%)
updating: models_doc2vec/autos_d2v_concat_2_3_0.1.model (deflated 7%)
updating: models_doc2vec/autos_d2v_pv-dm_2_3_0.1.model (deflated 9%)


In [52]:
from google.colab import files
files.download("models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [53]:
files.download(score_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Assess the best Model

In [54]:
#best_model = 'autos_d2v_pv-dbow_3_4_0.1'
best_model = 'autos_d2v_concat_1_3_0.1'

In [55]:
model = models[f"{best_model}.model"]

#### Assess self-similarity

In [56]:
ranks = []
second_ranks = []
similarities = []

train_corpus = data[f"{best_model}.train"]

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    similarities.append(sims)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [57]:
import collections

counter = collections.Counter(ranks)
print(counter)

tunes_sim = counter[0]
tunes_not_sim = len(train_corpus) - counter[0]

print()
print(f"Tunes similar to themselves in first place: {tunes_sim}")
print(f"Tunes not similar in first place: {tunes_not_sim}")
print(f"% of tunes not similar to themselves in first place: {100 * tunes_not_sim/(tunes_sim + tunes_not_sim):.2f}%")

Counter({0: 1481, 1: 122, 2: 40, 3: 23, 4: 20, 5: 14, 6: 12, 7: 10, 11: 9, 8: 8, 10: 6, 9: 6, 19: 5, 14: 5, 29: 4, 16: 3, 21: 3, 36: 3, 12: 3, 43: 2, 26: 2, 15: 2, 13: 2, 23: 1, 160: 1, 211: 1, 89: 1, 187: 1, 31: 1, 56: 1, 18: 1, 51: 1, 41: 1, 30: 1, 52: 1, 37: 1, 84: 1, 260: 1, 79: 1, 25: 1, 74: 1, 59: 1, 27: 1, 71: 1, 250: 1, 28: 1, 77: 1, 22: 1, 185: 1, 35: 1})

Tunes similar to themselves in first place: 1481
Tunes not similar in first place: 331
% of tunes not similar to themselves in first place: 18.27%


In [58]:
print('Tunes that are similar to other tunes before they are similar to themselves:')
for tune_index, value in enumerate(ranks):
  if value > 0:
    print(f"'{titles.iloc[tune_index, 0]}': {value}")

Tunes that are similar to other tunes before they are similar to themselves:
'A Blossom Fell': 1
'A Certain Smile': 1
'A Lovely Way To Spend An Evening': 1
'A Nightingale Sang In Berkeley Square': 1
'A Sleepin' Bee': 8
'A Tisket A Tasket': 1
'A Weaver Of Dreams': 5
'Ac-Cent-Tchu-Ate The Positive': 2
'After You': 1
'Afternoon In Paris': 1
'Alexander's Ragtime Band': 1
'All Alone': 2
'All God's Chillun Got Rhythm': 4
'All Of Me': 1
'All Through The Day': 1
'Almost Like Being In Love': 4
'Among My Souvenirs': 10
'April In Paris': 2
'As Long As I Live': 1
'At Last': 6
'Bags' Groove': 4
'Barbados': 5
'Barbara': 1
'Basin Street Blues': 1
'Baubles, Bangles and Beads': 2
'Bessie's Blues': 4
'Best Things In Life Are Free, The': 5
'Between The Devil And The Deep Blue Sea': 1
'Beyond The Sea': 1
'Bill Bailey': 8
'Billie's Bounce': 1
'Birth Of The Blues, The': 1
'Black Narcissus': 1
'Blue Monk': 3
'Blue Room, The': 2
'Blue Silver': 1
'Blue Skies': 1
'Blue Sphere': 4
'Blue Turning Grey Over You': 8

#### Determine Similarity of Tunes

In [59]:
assert(len(model.dv.vectors == len(list(title_to_index.keys()))))

In [60]:
tunes_eval_list = [
                   'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'Hundred Years From Today, A',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

In [61]:
for tune in tunes_eval_list:
  sim = model.dv.most_similar(title_to_index[tune])

  print()
  print(f"'{tune}' is most similar to:")
  for index, value in sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')



'Sweet Sue, Just You' is most similar to:
	0.876: Call Me
	0.793: Mack The Knife
	0.779: My Secret Love
	0.774: Pent Up House
	0.751: When My Dreamboat Comes Home
	0.744: Smile
	0.738: Three Little Words
	0.734: Begin The Beguine
	0.732: Breeze And I, The
	0.729: Strangers In The Night

'These Foolish Things' is most similar to:
	0.787: I Can't Get Started
	0.701: Summertime
	0.681: Mas Que Nada
	0.677: I've Found A New Baby
	0.664: Carry Me Back To Old Virginny
	0.660: This Is All I Ask
	0.659: Soul Eyes
	0.658: Doctor Jazz
	0.647: I Don't Want To Miss Mississippi
	0.634: On The Sunny Side Of The Street

'Blue Moon' is most similar to:
	0.728: Daybreak
	0.665: Avalon
	0.633: Making Whoopee
	0.615: Pagan Love Song
	0.614: Flamingo
	0.608: African Queen, The
	0.586: Wouldn't It Be Loverly
	0.558: Blue Tahitian Moon
	0.555: Well You Needn't
	0.553: Ill Wind

'Hundred Years From Today, A' is most similar to:
	0.798: Hallelujah I Just Love Him So
	0.776: Blue Turning Grey Over You
	0.736:

#### Evaluate Similarity for Jazz Contrafacts



In [62]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'doc2vec']
df_eval_doc2vec = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  row.append(model.dv.similarity(tune1, tune2))
  
  df_eval_doc2vec.loc[len(df_eval_doc2vec)] = row

df_eval_doc2vec

,Tune1,Tune2,doc2vec
0,26-2,Confirmation,-0.202874
1,52nd Street Theme,I Got Rhythm,0.383187
2,Ablution,All The Things You Are,0.904533
3,Anthropology,I Got Rhythm,0.324972
4,Ballade,As Long As I Live,0.465201
5,Bright Mississippi,Sweet Georgia Brown,0.816182
6,C.T.A.,I Got Rhythm,0.228300
7,Celia,I Got Rhythm,-0.146540
8,Cottontail,I Got Rhythm,0.630977
9,Countdown,Tune Up,0.278289


##### Compare results from TF-IDF and doc2vec



In [63]:
df_eval_tfidf.merge(df_eval_doc2vec)

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4,doc2vec
0,26-2,Confirmation,0.568533,0.564252,0.574305,0.576989,-0.202874
1,52nd Street Theme,I Got Rhythm,0.832439,0.832439,0.825817,0.817865,0.383187
2,Ablution,All The Things You Are,0.886735,0.877070,0.879810,0.878596,0.904533
3,Anthropology,I Got Rhythm,0.777413,0.777413,0.784389,0.783179,0.324972
4,Ballade,As Long As I Live,0.634909,0.643877,0.647502,0.641000,0.465201
5,Bright Mississippi,Sweet Georgia Brown,0.970778,0.969760,0.968172,0.965051,0.816182
6,C.T.A.,I Got Rhythm,0.698455,0.695247,0.704302,0.709436,0.228300
7,Celia,I Got Rhythm,0.505318,0.489912,0.486457,0.473649,-0.146540
8,Cottontail,I Got Rhythm,0.915579,0.915579,0.912508,0.909247,0.630977
9,Countdown,Tune Up,0.744414,0.725241,0.712231,0.697034,0.278289


##### Verbose Testing Output

In [64]:
similarity_found = 0

for tune1, tune2 in contrafacts.items():
  index1 = title_to_index[tune1]
  index2 = title_to_index[tune2]

  sim = model.dv.similarity(index1, index2)
  most_sim = model.dv.most_similar(index1, topn=5)
  
  # check whether the index2 was found among the topn similar tunes
  for i in most_sim:
    if index2 == i[0]:
      similarity_found += 1

  print()
  print(f"'{tune1}' to '{tune2}': {sim}")
  print(f"'{tune1}' is most similar to:")
  for index, value in most_sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')


print()
num_tunes = len(contrafacts)
print(f"Number of tunes tested: {num_tunes}")
print(f"Number of similarities found: {similarity_found}")
print(f"Result: {100 * similarity_found / num_tunes}%")


'26-2' to 'Confirmation': -0.2028740793466568
'26-2' is most similar to:
	0.739: St. Thomas
	0.718: 500 Miles High
	0.694: Whisper Not
	0.690: Cheryl
	0.682: Perhaps

'52nd Street Theme' to 'I Got Rhythm': 0.3831866383552551
'52nd Street Theme' is most similar to:
	0.904: Affirmation
	0.887: I've Got My Fingers Crossed
	0.827: Shutterbug
	0.816: A Felicidade
	0.811: Little Sunflower

'Ablution' to 'All The Things You Are': 0.9045333862304688
'Ablution' is most similar to:
	0.905: All The Things You Are
	0.785: On The Sunny Side Of The Street
	0.747: Sippin' At Bells
	0.740: I Didn't Know What Time It Was
	0.696: While We're Young

'Anthropology' to 'I Got Rhythm': 0.3249718248844147
'Anthropology' is most similar to:
	0.964: Salt Peanuts
	0.955: Moose The Mooche
	0.944: Shaw 'Nuff
	0.942: Oleo
	0.930: No Moe

'Ballade' to 'As Long As I Live': 0.46520131826400757
'Ballade' is most similar to:
	0.783: Tickle-Toe
	0.764: Lazy Afternoon
	0.763: Agua De Beber
	0.740: With A Song In My Hear

In [65]:
model.dv.similarity(0, 1)

0.71779776

#### Download Model

In [66]:
from google.colab import files
files.download(f"{model_path}/{best_model}.model")
files.download(f"{data_path}/{best_model}.train")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>